# Adjusted Cohort Graduation Rate

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from swampy import structshape as ss
import boto3
import shelve


In [ ]:
def make_raw_gr_frame(year):
    """Create raw graduation rate dataframe for a given year from S3. It is 'raw' because
    no modifications are done and it is cached locally as-is."""
    conn = boto3.client('s3')

    # Graduation rate raw data filenames on S3
    year_to_loc = {2018: "grad_rate/acgr-sch-sy2018-19-wide.csv",
                   2017: "grad_rate/acgr-sch-sy2017-18.csv",
                   2016: "grad_rate/acgr-sch-sy2016-17.csv",
                   2015: "grad_rate/acgr-sch-sy2015-16.csv",
                   2014: "grad_rate/acgr-release2-sch-sy2014-15.csv",
                   2013: "grad_rate/acgr-sch-sy2013-14.csv",
                   2012: "grad_rate/acgr-sch-sy2012-13.csv",
                   2011: "grad_rate/acgr-sch-sy2011-12.csv",
                   2010: "grad_rate/acgr-sch-sy2010-11.csv"}

    # Verify input parameter is valid
    if year not in list(range(2010, 2019)):
        raise ValueError("input parameter {} is out of range.".format(year))

    # Local storage (cache) for the raw data so that iteration time is faster.
    shelf = shelve.open("gr_dfs")

    df = pd.DataFrame()
    shelf_key = str(year)

    if shelf_key in shelf:
        df = shelf[shelf_key]
    else:
        tmp_file_name = 'curr_file.csv.bak'
        conn.download_file('edu-data-bucket', year_to_loc[year], tmp_file_name)

        df = pd.read_csv(tmp_file_name)
        shelf[shelf_key] = df

        if os.path.exists(tmp_file_name):
            os.remove(tmp_file_name)

    # Shelve teardown
    shelf.close()
    return df


def year_string(y: int):
    """Input an integer year and get a range that matches the column suffixes in the raw data.
    e.g. 2011 => 1112 and 2018 => 1819."""
    return str(y)[-2:] + str(int(str(y)[-2:]) + 1)


In [ ]:
dfs = [make_raw_gr_frame(year=y) for y in range(2010, 2019)]
years = [year_string(y) for y in range(2010, 2019)]

# Many of the column names have the school year in them, which presents a challenge for combining all the years
# into one large dataframe.


shape_data = [(school_year, df.shape) for school_year, df in zip(years, dfs)]
shape = pd.DataFrame(shape_data, columns=('school_year', 'shape'))
shape

# Inspect features that are present in some but not common to all.
# Start by removing the years from the column names.
# cols_wo_year => Column names without the year
cols_wo_year = [list(map(lambda x: x.replace(y, ""), df.columns))
                for y, df in zip(years, dfs)]
print(set(cols_wo_year[3]) - set(cols_wo_year[0]))
print(set(cols_wo_year[6]) - set(cols_wo_year[0]))
print(set(cols_wo_year[7]) - set(cols_wo_year[0]))
print(set(cols_wo_year[8]) - set(cols_wo_year[0]))


In [ ]:
# INSERT_DATE refers to when the data was inserted and is not relevant for our study.
dfs[3].drop(['INSERT_DATE'], axis=1, inplace=True)
# ST_SCHID and ST_LEAID are values assigned by the state which are not found in the other years. From the data, it looks like maybe these
# started being assigned in 2016. If we need another geographical grouping mechanism in the future we can look into it.
dfs[6].drop(['ST_LEAID', 'ST_SCHID'], axis=1, inplace=True)
# HOM_COHORT and FCS_COHORT refer to the subpopulation of homeless and foster care students, which was not tracked before school year 2017-2018
idx7_sr = shape.school_year[7]
idx8_sr = shape.school_year[8]
dfs[7].drop(['ST_LEAID', 'ST_SCHID', 'FCS_RATE_'+idx7_sr, 'FCS_COHORT_'+idx7_sr,
            'HOM_RATE_'+idx7_sr, 'HOM_COHORT_'+idx7_sr], axis=1, inplace=True)
dfs[8].drop(['ST_LEAID', 'ST_SCHID', 'FCS_RATE_'+idx8_sr, 'FCS_COHORT_'+idx8_sr,
            'HOM_RATE_'+idx8_sr, 'HOM_COHORT_'+idx8_sr], axis=1, inplace=True)


In [ ]:
# Create large df of all the years
cols_wo_school_year = [list(map(lambda x: x.replace(y, ""), df.columns))
                       for y, df in zip(years, dfs)]
for num1, num2 in zip(range(0, 8), range(1, 9)):
    assert cols_wo_school_year[num1] == cols_wo_school_year[num2]

big_df = pd.DataFrame()
print("big_df_columns", big_df.columns)
for idx, df in enumerate(dfs):
    df.columns = cols_wo_school_year[0]
    df['Year'] = list(range(2010, 2019))[idx]
    # reorder columns to be how we want
    df = df[['Year']+cols_wo_school_year[0]]
    big_df = pd.concat([big_df, df], axis=0)


big_df.shape, big_df.head(n=3)


In [ ]:
# Missingness
big_df.isnull().sum()

# Why doesn’t the summation of the major racial and ethnic groups equal the “ALL” student count?
# * Due to flexibilities with states’ implementation of the Elementary and Secondary Education Act,
#   there may be instances where not all possible groupings of racial/ethnic identification are reported
#   as individual subgroups. Therefore, some information may be missing and these counts by major racial
#   and ethnic group will not include every student; however any students not included within an individual
#   major racial and ethnic group would be included in the “ALL” student count.
# #### Why are the major racial and ethnic groups reported differently by states?
# * Under the ESEA, a State educational agency (SEA) has the flexibility to determine the major racial/ethnic
#   groups it will use for reporting on the data included in its assessment and accountability system.  The
#   subgroups that an SEA uses are approved through its Accountability Workbook (the most recent copy of each
#   state’s workbook can be found here:  http://www2.ed.gov/admins/lead/account/stateplans03/index.html).
#   As a result, there is some variation in how SEAs report data by race and ethnicity.

# Drop population subgroup columns
cols_to_keep = ['Year', 'STNAM', 'FIPST', 'LEAID', 'LEANM',
                'NCESSCH', 'SCHNAM', 'ALL_COHORT_', 'ALL_RATE_']
big_df = big_df[cols_to_keep]


# Drop rows where ALL_COHORT_ == '.'
instances_after_drop = len(big_df) - 2913
# Reindex the dataframe because their are duplicate indexes
big_df.index = list(range(len(big_df)))
big_df.drop(big_df[big_df.ALL_COHORT_ == '.'].index, axis=0, inplace=True)
assert len(big_df) == instances_after_drop
print("Number of instances after this drop", len(big_df))
# Can convert ALL_COHORT_ column to numeric now
big_df.loc[:, 'ALL_COHORT_'] = np.int64(big_df.ALL_COHORT_.values)
big_df.dtypes

# Drop rows where ALL_RATE_ == 'PS'. About 1400 schools per year are dropped.
# Approximately 1200 schools per year were dropped that had a cohort size of 5 or less 
# because they had no graduation rate.
# Outliers seen during ingestion of the individual Adjusted Cohort Dataset had cohort sizes of 5-30,000 (3 schools).
# These corresponded to one large virtual high school in Ohio and two schools with listed cohort sizes on their website of ~150 (transcription errors).

print("Dropping", len(big_df[big_df.ALL_RATE_ == 'PS']))
big_df.drop(big_df[big_df.ALL_RATE_ == 'PS'].index, axis=0, inplace=True)
print("Number of instances after this drop", len(big_df))


In [ ]:
# Several columns are of object type, which means that either text or Nan values are present.
def print_object_column_info(df):
    """
    Print column data types and the number of object columns.
    """
    print(big_df.dtypes, "{} object columns present". format(
        len(big_df.select_dtypes(include=object).count())))


print_object_column_info(big_df)


In [ ]:
# Convert year feature to numeric
big_df.loc[:, 'Year'] = np.int64(big_df.Year)

# THe ALL_RATE_ column has numbers given as ranges, numbers with GE,LE,LT,GT prefixes, and also 'PS'.
# The codebook explains that these are to conceal specifics when it would give us the ability to identify individual students. The 'PS' specifically means that the cohort had 5 or fewer students in it.
# Drop the ALL_RATE_ rows that have 'PS' and convert the rest to the average meaning of the term. E.g. GT50 => 75, LT50 => 25, 88-92 => 90.

# Helper functions to convert ranges given in ALL_RATE_ column to a trivially convertible string


def conv_range_to_numeric_string(t: str):
    """Take in a numeric range given as a string, e.g. "10-20" and return the midpoint as a string."""
    vals = t.split("-")
    if len(vals) == 2:
        val1, val2 = float(vals[0]), float(vals[1])
        new_val = val1 / 2.0 + val2 / 2.0
        return str(new_val)
    else:
        # If a range with a dash in it is not found, return original string unchanged.
        return t


assert conv_range_to_numeric_string('95-95') == '95.0'
assert conv_range_to_numeric_string('105-110') == '107.5'
assert conv_range_to_numeric_string('90') == '90'


def strip_prefixed_string(t: str):
    """
    Convert prefixed numbers to a string of a number in the midpoint of their range 
    """
    if t.startswith('GT'):
        t = t.removeprefix('GT')
        t = float(t)
        t = sum([t, 100.0])/2.0
        return str(t)
    elif t.startswith('GE'):
        t = t.removeprefix('GE')
        t = float(t)
        t = sum([t, 100.0])/2.0
        return str(t)
    elif t.startswith('LT'):
        t = t.removeprefix('LT')
        t = float(t)
        t = sum([t, 0])/2.0
        return str(t)
    elif t.startswith('LE'):
        t = t.removeprefix('LE')
        t = float(t)
        t = sum([t, 0])/2.0
        return str(t)
    # If one of the prefixes is not found, return original string unchanged
    return t


assert strip_prefixed_string('GE50') == '75.0'
assert strip_prefixed_string('GT50') == '75.0'
assert strip_prefixed_string('LE90') == '45.0'
assert strip_prefixed_string('LT90') == '45.0'
assert strip_prefixed_string('95-95') == '95-95'

# Run the above functions on the series to produce a float64 series
big_df.loc[:, 'ALL_RATE_'] = big_df.ALL_RATE_.map(
    strip_prefixed_string, na_action='ignore')
big_df.loc[:, 'ALL_RATE_'] = big_df.ALL_RATE_.map(
    conv_range_to_numeric_string, na_action='ignore')
big_df.loc[:, 'ALL_RATE_'] = pd.to_numeric(big_df.ALL_RATE_.tolist())


In [ ]:
large_school_indexes = big_df.ALL_COHORT_ > 5000
big_df.loc[large_school_indexes, :]

# Schools are found with >20,000 students in a cohort. Based on google search, the graduating classes are closer to 100-200 than the extremely large sizes reported.
# We drop these schools since they are called into question. Note that we could impute them with the mean school size.

big_df.drop(big_df[large_school_indexes].index, axis=0, inplace=True)
sns.histplot(big_df.ALL_COHORT_,)
plt.show()
big_df.ALL_COHORT_.describe()

sns.histplot(big_df.ALL_COHORT_, cumulative=True, stat='probability')

xlimits = np.percentile(big_df.ALL_COHORT_, [5, 95])
plt.title("Cohort size, x range = 5-95%")
plt.xlim(xlimits)
plt.grid(visible=True)


In [ ]:
# Rename BUREAU OF INDIAN AFFAIRS to BUREAU OF INDIAN EDUCATION
indian_affairs_rows = big_df.query(
    "STNAM.str.contains('INDIAN AFFAIRS')").index
big_df.loc[indian_affairs_rows, 'STNAM'] = 'BUREAU OF INDIAN EDUCATION'


In [ ]:
big_df.to_csv('grad_rate_intermediate.csv.bak', index=False)


# School Directory Dataset

In [ ]:
conn = boto3.client('s3')
local_dir_filename = 'directory_all.csv.bak'
if not os.path.exists('./' + local_dir_filename):
    conn.download_file(
        'edu-data-bucket', 'directory/schools_ccd_directory.csv', local_dir_filename)

df0 = pd.read_csv(local_dir_filename, low_memory=False)

df0.shape


#### There are many years and schools in this dataset that are not in the graduation rate dataset
    * Ncessch and ncessch_num are duplicates
    * Features shared with the Adjusted Cohort dataset are
        * ncessch
        * school_name
        * leaid
        * lea_name
        * state_location
        * fips

In [ ]:
# Subset to our years of interest
years = list(range(2010, 2019))
our_years = df0.query('year in @years')
pd.set_option('display.max_columns', 99)
our_years.head(n=3)

# Remove uninteresting columns
# Uninteresting because duplicate: 'school_id', 'ncessch_num', 'leaid','lea_name','school_name', 'state_location','fips', 'phone_string'
# Verify ncessch == ncescch_num since we will merge on it later.
# Leave school district identifier for verification that the merge worked later.
assert all(our_years.ncessch == our_years.ncessch_num)
our_years.drop(['ncessch_num', 'school_id', 'lea_name', 'state_location', 'fips', 'school_status', 'longitude', 'phone', 'city_location', 'street_location', 'street_mailing', 'state_leaid', 'state_leg_district_lower', 'state_leg_district_upper', 'congress_district_id', 'csa', 'cbsa', 'direct_certification', 'title_i_schoolwide', 'lunch_program', 'seasch', 'city_mailing', 'state_mailing', 'elem_cedp', 'middle_cedp', 'high_cedp', 'shared_time', 'bureau_indian_education'], axis=1, inplace=True)


In [ ]:
# Meaningful Missingness
# Use school directory cookbook to make decisions are removing columns that we are uncertain about.
# Of the schools with missing virtual identifier, we are not able to find any that contain obious names that 
# would indicate they are virtual. Knowing that virtual schools are less common than brick and mortal, we 
# impute the missingess as a False value.

assert len(our_years.query('virtual < 0 and "Virtual" in school_name')) == 0
assert len(our_years.query('virtual < 0 and "virtual" in school_name')) == 0
assert len(our_years.query('virtual < 0 and "Correspondence" in school_name')) == 0
assert len(our_years.query('virtual < 0 and "correspondence" in school_name')) == 0
assert len(our_years.query('virtual < 0 and "Electronic" in school_name')) == 0
assert len(our_years.query('virtual < 0 and "electronic" in school_name')) == 0
assert len(our_years.query('virtual < 0 and "digital" in school_name')) == 0
assert len(our_years.query('virtual < 0 and "Digital" in school_name')) == 0

# This line saves an additional 20K instances and improves the model predictions by about 0.04 accuracy
our_years.loc[:, 'virtual'] = our_years.virtual.replace(np.nan, 0)
assert our_years.virtual.isnull().sum() == 0


In [ ]:
# Missingness

# Impute -1,-2,-3 missing values to nans
# Find strings equal to -1,-2, or -3
special_vals = pd.DataFrame([(our_years.select_dtypes(include=[object]) == "-1").sum(), (our_years.select_dtypes(
    include=[object]) == "-2").sum(), (our_years.select_dtypes(include=[object]) == "-3").sum()])
special_vals.index = ["-1", "-2", "-3"]
special_vals

special_vals = pd.DataFrame([(our_years.select_dtypes(include=[np.number]) == -1).sum(), (our_years.select_dtypes(
    include=[np.number]) == -2).sum(), (our_years.select_dtypes(include=[np.number]) == -3).sum()])
special_vals.index = ["-1", "-2", "-3"]
special_vals

str_cols = our_years.select_dtypes(include=object).columns.tolist()
num_cols = our_years.select_dtypes(include=np.number).columns.tolist()
str_col_special_nans = (our_years[str_cols] ==
                        "-1") | (our_years[str_cols] == "-2")
num_col_special_nans = (our_years[num_cols] < 0)
assert "highest_grade_offered" in num_cols

our_years.loc[:, str_cols] = our_years[str_cols].where(
    ~str_col_special_nans, other=np.nan)
our_years.loc[:, num_cols] = our_years[num_cols].where(
    ~num_col_special_nans, other=np.nan)

our_years.highest_grade_offered.value_counts()
non_high_schools = our_years.query('highest_grade_offered <= 9')
our_years.drop(non_high_schools.index, axis=0, inplace=True)


In [ ]:
import missingno
missingno.matrix(our_years)


In [ ]:
# Examine reduced_price_lunch to see how similar it is to free_or_reduced_price_lunch
np.subtract(our_years.free_lunch, our_years.free_or_reduced_price_lunch)
# It looks like there is some additional information in both columns so we'll keep them both.


In [ ]:
# Drop all rows with a missing feature, the simplest way to get rid of our nans
our_years.dropna(inplace=True)
our_years = our_years.convert_dtypes(convert_string=False)


In [ ]:
# Examine features that are still objects
pd.DataFrame(our_years.dtypes).T


In [ ]:
our_years.shape
missingno.matrix(our_years)


Look at # FTE Teachers and Enrollment and Free or Reduced Price Lunch

In [ ]:
# Enrollment less than 1 does not make any sense.
# Teachers less than 1 does not make much sense. It is possible that we have part time teachers, but we
# are already dropping schools with less than 5 in their graduating cohort. The decision is made to
# drop schools with less than 1 teacher as well. There is likley significant overlap between these and the
# ALL_COHORT == 'ps' schools.
# @NOTE: The schools definitely warrant further study to help understand what is occuring there.
# There is over evidence below that the Common Core of Data School Directory dataset is a fairly messy dataset.
print(our_years.query('enrollment < 1 or teachers_fte < 1')
      ['county_code'].count())
print(our_years.query('reduced_price_lunch > enrollment')
      ['county_code'].count())
print(our_years.query('free_or_reduced_price_lunch > enrollment')
      ['county_code'].count())

our_years.drop(our_years.query(
    'enrollment < 1 or teachers_fte < 1').index, inplace=True)
our_years.drop(our_years.query(
    'reduced_price_lunch > enrollment').index, inplace=True)
our_years.drop(our_years.query(
    'free_or_reduced_price_lunch > enrollment').index, inplace=True)

print(our_years.query('enrollment < 1 or teachers_fte < 1')
      ['county_code'].count())
print(our_years.query('reduced_price_lunch > enrollment')
      ['county_code'].count())
print(our_years.query('free_or_reduced_price_lunch > enrollment')
      ['county_code'].count())


In [ ]:
# A small number of schools have a highest_grade_offered less than 12.
# Looking up several of these schools
# * Hobbs Freshman High School is a 9th grade only school
# * Cyberacademy of South Carolina is K-12
# * With this mixed information, we are opting to drop the rows.
our_years.drop(our_years.query(
    'highest_grade_offered < 12').index, inplace=True)


print(our_years.teachers_fte.describe())
print(our_years.highest_grade_offered.describe())


In [ ]:
our_years.info()
our_years.to_csv('directory_intermediate_dataframe.csv.bak', index=False)
our_years.head(n=3)


# Math and Reading / Language Arts Assessment Participation

In [ ]:
# Ingest from S3
conn = boto3.client('s3')
local_dir_filename = 'math_rla_percent_participation_prefiltered.csv.bak'
if not os.path.exists('./' + local_dir_filename):
    conn.download_file(
        'edu-data-bucket', 'percent_participation/' + local_dir_filename, local_dir_filename)


perc_part = pd.read_csv(local_dir_filename, low_memory=False)
perc_part.shape, perc_part.info()

# Drop rows where Math_Pct_Part == 'PS'.
print("Dropping", len(perc_part[perc_part.Math_Pct_Part == 'PS']))
print("Dropping", len(perc_part[perc_part.Rla_Pct_Part == 'PS']))
perc_part.drop(perc_part[perc_part.Math_Pct_Part ==
               '.'].index, axis=0, inplace=True)
perc_part.drop(perc_part[perc_part.Rla_Pct_Part ==
               '.'].index, axis=0, inplace=True)
perc_part.drop(perc_part[perc_part.Math_Pct_Part ==
               'PS'].index, axis=0, inplace=True)
perc_part.drop(perc_part[perc_part.Rla_Pct_Part ==
               'PS'].index, axis=0, inplace=True)
print("Number of instances after this drop", len(big_df))

# Run the above functions on the series to produce a float64 series
perc_part.loc[:, 'Math_Pct_Part'] = perc_part.Math_Pct_Part.map(
    strip_prefixed_string, na_action='ignore')
perc_part.loc[:, 'Math_Pct_Part'] = perc_part.Math_Pct_Part.map(
    conv_range_to_numeric_string, na_action='ignore')
perc_part.loc[:, 'Math_Pct_Part'] = pd.to_numeric(
    perc_part.Math_Pct_Part.tolist())

perc_part.loc[:, 'Rla_Pct_Part'] = perc_part.Rla_Pct_Part.map(
    strip_prefixed_string, na_action='ignore')
perc_part.loc[:, 'Rla_Pct_Part'] = perc_part.Rla_Pct_Part.map(
    conv_range_to_numeric_string, na_action='ignore')
perc_part.loc[:, 'Rla_Pct_Part'] = pd.to_numeric(
    perc_part.Rla_Pct_Part.tolist())


# Save intermediate file locally
perc_part.to_csv("percent_participation_intermediate.csv.bak", index=False)


# Merge datasets and output for exploration and modeling 

In [ ]:
"""
Merge intermediate dataframes 
"""
gr = pd.read_csv('grad_rate_intermediate.csv.bak')
directory = pd.read_csv('directory_intermediate_dataframe.csv.bak')
perc_part = pd.read_csv('percent_participation_intermediate.csv.bak')

perc_part


In [ ]:

merge0 = gr.merge(directory, how='left', left_on=(
    'Year', 'NCESSCH'), right_on=('year', 'ncessch'))
merge1 = merge0.merge(perc_part, how='left', left_on=(
    'Year', 'NCESSCH'), right_on=('Year', 'NCESSCH'))
merge1.head()


In [ ]:
# Drop redundant columns seen after merging
merge1.drop(['year', 'ncessch', 'leaid'], axis=1, inplace=True)
merge1.dropna(inplace=True)
merge1.shape

# Reorder columns
merge1 = merge1[[
    i for i in merge1.columns if i not in 'ALL_RATE_'] + ['ALL_RATE_']]


In [ ]:
missingno.matrix(merge1)


In [ ]:
# View column pairs with minimal positive correlation or above
pd.DataFrame((np.abs(merge1.corr()) > 0.3).sum()).T


In [ ]:
# Save out dataset A to disk
merge1.to_csv('../data/mergeA.csv', index=False)


# County unemployment rates from USDA 

In [ ]:
# For dataset B, we are adding some new data to attempt to capture whatever underlying factors county_code and state ID's were 
# being selecting for in the first round of modeling.
# This way, the model is not predicting performance from past years, and also is not predicting a certain fate based on the county
# or state ID itself.
# Source: https://www.ers.usda.gov/data-products/county-level-data-sets/ 

In [ ]:
import pandas as pd
county_raw = pd.read_excel("~/datasets/county_usda_data_2000_2020/Unemployment_Median_Income.xlsx",header=4)
# Un-needed columns at end of the frame
county_raw.drop(['Median_Household_Income_2020','Med_HH_Income_Percent_of_State_Total_2020'],axis=1,inplace=True)

In [ ]:
pd.set_option("display.max_columns",990)
county_raw.tail(n=3)

In [ ]:
county_reshaped = pd.DataFrame(columns=['year','county_code','area_name','rural_urban_continuum_code','urban_influence_code','metro_or_not','civilian_labor_force','unemployment_rate'])
for (row_id,series) in county_raw.iterrows():
    county_code = series[0]
    area_name = series[2]
    rural_urban_continuum_code = series[3] 
    urban_influence_code = series[4]
    metro_or_not = series[5]
    years = list(range(2000,2022))
    id_starts = list(range(6, len(county_raw.columns),4))
    for idx, year in enumerate(years):
        civilian_labor_force =  series[id_starts[idx]]
        unemployment_rate = series[id_starts[idx]+3]
        if year < 2010 or year > 2018:
            continue
        county_reshaped.loc[len(county_reshaped.index)] = year, county_code, area_name, rural_urban_continuum_code,urban_influence_code,metro_or_not,civilian_labor_force,unemployment_rate
        # print(year, civilian_labor_force,unemployment_rate)
        # Tail and head of the dataset show that these fields are being pulled correctly

In [ ]:
county_reshaped.tail(n=25)

In [ ]:
merge1.info()

In [ ]:
merge2 = merge1.merge(county_reshaped,how='left',left_on=['Year','county_code'],right_on=['year','county_code'])

In [ ]:
merge2.isnull().sum()

In [ ]:
assert merge2.Year.equals(merge2.year)
merge2.drop('year',axis=1,inplace=True)

In [ ]:
# Visually validate that the school district names appear to match the area names for the counties. 
# These are from two different datasets so we are making sure the two are self-consitent.
merge2[['LEANM','area_name']]

In [122]:
# Save out dataset B to disk
merge2.to_csv('../data/mergeB.csv', index=False)